<div class="alert alert-block alert-info"><b>Genshin Impact buff optimization thingy</b></div>

General idea of this thing is this: main widget assembles the party out of the hypercarry and buffer widgets. Those widgets handle whatever internal calculations for the members.
Main widget asks hypercarry to calculate their 'solo' stats, ask buffers to calculate their buffs, adds the buffs, asks hypercarry to apply conversions, and calculates the damage output.
Optimizers should assemble either relevant buffers or relevant artifacts and call the 'calculate everything' function.
As such, either the calculator should be separate, or the optimizers should be parts of the main widget. The former seems to be a better option.

<b>To do</b>:
- DONE Rework hypercarry class stats
- DONE Add specific hypercarries such as Raiden, Hu Tao, Itto and make separate hypercarry widget
- DONE Add Chevreuse as a buffer (HP scale + VV)
- DONE Add VV as a buffer (with average gained value being ~25%)
- DONE Rework buffer class to allow better constellation tracking maybe
- DONE Add input validator for numerical fields: https://stackoverflow.com/questions/15829782/how-to-restrict-user-input-in-qlineedit-in-pyqt , or change them for QSpinBox
- DONE Redo talent selectors for QSpinBox
- DONE Inheritance for character widgets
- DONE Redo the whole thing as 'party composition' class containing a HC and buffers to allow resonances to be smoother
- DONE Add Zhongli as a buffer
- DONE Add resonances and a resonator
- DONE Implement cycling resonator's elements for the buffer optimizer
- DONE Add frames for the pretty
- DONE Make the calculator and optimisers into their proper widgets
- DONE Add weapons as separate widgets
- DONE Add relevancy lists for buffers and resonances
- Add more weapons: Homa
- Add more characters: Xiao, Faruzan, Ayato
- Rework weapon-character link (?)
- Rework how character/weapon widget work -> universal widget class, list of fields to display in an entity
- Rework hypercarry stat keys for uniformity, including elements

In [1]:
# To adapt Qt4 code: QtGui -> QtWidgets
from PyQt5 import QtWidgets, QtCore

from itertools import product, combinations

In [2]:
# Utility functions and placeholders
class Char_placeholder():
    def __init__(self, name='None', element='None'):
        self.name = name
        self.element = element
        
class Widget_char_placeholder(QtWidgets.QWidget):
    def __init__(self, char_type=Char_placeholder):
        super(Widget_char_placeholder, self).__init__()
        self.character = char_type()
        
def make_basic_frame(layout):
    frame = QtWidgets.QFrame()
    frame.setLayout(layout)
    frame.setFrameStyle(QtWidgets.QFrame.StyledPanel)
    frame.setLineWidth(3)
    return frame

def make_check_box(caller, options, default_v, c_func=None):
    box = QtWidgets.QCheckBox(options, caller)
    box.setChecked(default_v)
    if c_func: box.stateChanged.connect(c_func)
    return box

def make_combo_box(caller, options, default_v, c_func=None):
    box = QtWidgets.QComboBox(caller)
    box.addItems(options)
    box.setCurrentText(default_v)
    if c_func: box.activated.connect(c_func)
    return box
        
def make_spin_box(caller, options, default_v, c_func=None):
    box = QtWidgets.QSpinBox(caller)
    box.setMinimum(options[0])
    box.setMaximum(options[1])
    box.setValue(default_v)
    if c_func: box.valueChanged.connect(c_func)
    return box

def make_double_spin_box(caller, options, default_v, c_func=None):
    box = QtWidgets.QDoubleSpinBox(caller)
    box.setMinimum(options[0])
    box.setMaximum(options[1])
    box.setValue(default_v)
    if c_func: box.valueChanged.connect(c_func)
    return box

def make_bold_label(caller, options, default_v, c_func=None):
    box = QtWidgets.QLabel(default_v, caller)
    box.setStyleSheet("font-weight: bold")
    return box

def make_labled_box(caller, name, type, options, default_v, c_func=None):
    label = QtWidgets.QLabel(name + ':' if name != '-' and type != 'text' else name, caller) if name != None else None
    box = {'bool': make_check_box, 'str': make_combo_box, 'int': make_spin_box, 'double': make_double_spin_box, 'bold': make_bold_label}[type](caller, options, default_v, c_func) if type != 'text' else None
    return {'label': label, 'box': box}

In [3]:
# Calculators
def res_mult(res):
    # Using the formulas from the wiki
    if res > 75:
        return 1/(4*res/100 + 1)
    elif res > 0:
        return 1 - res/100
    else:
        return 1 - res/200
        
def get_elements_in_party(hypercarry, buffers):
    elements = [hypercarry.element]
    for buffer in buffers:
        if buffer.element != 'None': elements.append(buffer.element)
    return elements        

def calculate_output(enemy_res, hypercarry, weapon=None, artifacts=None, buffers=None):
    if weapon == None:
        weapon = hypercarry.weapon
    stats = hypercarry.calculate_stats(weapon = weapon, artifacts = artifacts)
    stats['elements_in_party'] = get_elements_in_party(hypercarry, buffers)
    # Elemental resonances - the ones that count are Pyro, Hydro, Cryo, Geo and Dendro
    if stats['elements_in_party'].count('Pyro') >= 2:
        stats['ATK'] = stats['ATK'] + 0.25*hypercarry.base_atk(weapon)
    if stats['elements_in_party'].count('Hydro') >= 2:
        stats['HP'] = stats['HP'] + 0.25*hypercarry.base_stats['HP']
    if stats['elements_in_party'].count('Cryo') >= 2:
        stats['CV'] = stats['CV'] + 30 # Not really true for Melt, but whatever
    if stats['elements_in_party'].count('Geo') >= 2:
        stats['DMG'] = stats['DMG'] + 15
        if hypercarry.element == 'Geo': enemy_res = enemy_res - 20
    if stats['elements_in_party'].count('Dendro') >= 2:
        stats['EM'] = stats['EM'] + 75 # Average since we don't know the reactions
    # Buffs
    stats = hypercarry.apply_self_buffs(stats)
    stats = weapon.apply_buffs(stats)
    for buffer in buffers:
        if buffer.name != 'None': buffer.buff(stats, hypercarry, weapon)
    for shred in ['shred_vv', 'shred_chevreuse', 'shred_zhongli']:
        if shred in stats: enemy_res = enemy_res - stats[shred]
    # Conversions
    stats = hypercarry.apply_conversions(stats)
    stats = weapon.apply_conversions(stats)
    stats['CR'] = min(stats['CV']/4, 100)
    stats['CD'] = stats['CV'] - 2*stats['CR']
    return stats['ATK']*(1 + stats['DMG']/100)*(1 + (stats['CR']/100)*(stats['CD']/100))*res_mult(enemy_res), stats
    
def format_stats(stats):
    stats_formatted = ''
    shred_sum = 0
    for key, value in stats.items():
        if key in ('HP', 'ATK', 'DEF', 'EM'): stats_formatted = stats_formatted + key + ': ' + "{:.0f}".format(value) + ' | '
        if key in ('ER', 'DMG', 'CR', 'CD'): stats_formatted = stats_formatted + key + ': ' + "{:.1f}".format(value) + '% | '
        if 'shred' in key: shred_sum = shred_sum + stats[key]
    stats_formatted = stats_formatted + 'RES shred: ' + "{:.1f}".format(shred_sum)
    return stats_formatted

In [4]:
# Base classes
class Char_hypercarry(Char_placeholder):
    def __init__(self, name='Basic hypercarry', element='Physical', scales_with=('ATK', 'DMG', 'CV')):
        super(Char_hypercarry, self).__init__(name, element)
        self.scales_with = scales_with # What stats do we need to show on the widget? Full list of options is  ('HP', 'ATK', 'DEF', 'EM', 'ER', 'DMG', 'CV')
        # Base stats: character ascension
        self.base_stats = {
            'HP': 10000,
            'ATK': 300,
            'DEF': 800,
            'asc_stat': 'None', # Base hypercarry does not have a relevant ascension stat
            'asc_value': 0
        }
        # Weapons
        self.weapon_types = {
            'Custom weapon': Widget_weapon(Weapon_custom, self)
        }
        self.weapon = self.weapon_types['Custom weapon'].weapon # Since this is a reference and not a copy, it can be direct like this instead of an index with same effect
        # Extra stats - artifact substats and non-accounted-for effects
        self.extra_stats = {
            'HP_flat': 4780, # From sources such as the flower or rare weapon effects
            'HP_perc': 0,
            'ATK_flat': 311,
            'ATK_perc': 0,
            'DEF_flat': 0,
            'DEF_perc': 0,
            'EM': 0,
            'ER': 0,
            'DMG': 0, # Ignoring the goblet mainstat, but including such things as artifact 2p effects
            'CV': 170 # Base level invested
        }
        # Artefact types that interest us for this hypercarry
        self.artifact_types = {
            'Sands': (['Other'] 
                            + (['HP%'] if 'HP' in self.scales_with else [])
                            + (['ATK%'] if 'ATK' in self.scales_with else [])
                            + (['DEF%'] if 'DEF' in self.scales_with else [])
                            + (['EM'] if 'EM' in self.scales_with else [])
                            + (['ER%'] if 'ER' in self.scales_with else [])),
            'Goblet': (['Other']
                            + (['HP%'] if 'HP' in self.scales_with else [])
                            + (['ATK%'] if 'ATK' in self.scales_with else [])
                            + (['DEF%'] if 'DEF' in self.scales_with else [])
                            + (['EM'] if 'EM' in self.scales_with else [])
                            + (['Elemental DMG%'] if 'DMG' in self.scales_with else [])),
            'Circlet': (['Other']
                            + (['HP%'] if 'HP' in self.scales_with else [])
                            + (['ATK%'] if 'ATK' in self.scales_with else [])
                            + (['DEF%'] if 'DEF' in self.scales_with else [])
                            + (['EM'] if 'EM' in self.scales_with else [])
                            + (['CRIT'] if 'CV' in self.scales_with else []))
        }
        self.artifacts = {
            'Sands': 'Other',
            'Goblet': 'Other',
            'Circlet': 'Other'
        }
        # Extras: to be filled on character basis
        self.fields = (
            ['Basic hypercarry has no special optional properties', 'text', None, None],
        )
        
    def base_atk(self, weapon=None):
        if weapon == None:
            weapon = self.weapon
        return self.base_stats['ATK'] + weapon.atk[weapon.level]
    # Note: this one function - or rather the requirement of the 'hypercarry base attack' value for both character and weapon buff calculations complicates things immensely
    # Either a weapon has to be an object referenced as property of a character - in which case it has to contain holder reference which isn't good practice,
    # or it has to be contained within the character object itself - in which case we can't override weapons in a convenient manner
    # Possible solution: rewrite it so that 'hypercarry widget' contains a character widget, a weapon widget, and calls buff functions itself, passing one to another when necessary
        
    def calculate_stats(self, weapon=None, artifacts=None):
        # Calculating 'internal' stats. Allows me to not have to change the calculate_output function for specific hypercarries
        stats = {}
        if weapon == None:
            weapon = self.weapon
        if artifacts == None: # No build override
            sands = self.artifacts['Sands']
            goblet = self.artifacts['Goblet']
            circlet = self.artifacts['Circlet']
        else: # Overriding artifacts with provided
            sands = artifacts[0]
            goblet = artifacts[1]
            circlet = artifacts[2]
        stats['HP'] = (1 
                       + (0.466 if sands == 'HP%' else 0) + (0.466 if goblet == 'HP%' else 0) + (0.466 if circlet == 'HP%' else 0) 
                       + self.extra_stats['HP_perc']/100)*self.base_stats['HP'] + self.extra_stats['HP_flat']
        stats['ATK'] = (1 
                        + (0.466 if sands == 'ATK%' else 0) + (0.466 if goblet == 'ATK%' else 0) + (0.466 if circlet == 'ATK%' else 0) 
                        + self.extra_stats['ATK_perc']/100)*self.base_atk(weapon) + self.extra_stats['ATK_flat']
        stats['DEF'] = (1 
                        + (0.583 if sands == 'DEF%' else 0) + (0.583 if goblet == 'DEF%' else 0) + (0.583 if circlet == 'DEF%' else 0) 
                        + self.extra_stats['DEF_perc']/100)*self.base_stats['DEF'] + self.extra_stats['DEF_flat']
        stats['EM'] = 0 + (186.5 if sands == 'EM' else 0) + (186.5 if goblet == 'EM' else 0) + (186.5 if circlet == 'EM' else 0) + self.extra_stats['EM']
        stats['ER'] = 100 + (51.8 if sands == 'ER%' else 0) + self.extra_stats['ER']
        stats['DMG'] = 0 + ((58.3 if self.element == 'Physical' else 46.6) if goblet == 'Elemental DMG%' else 0) + self.extra_stats['DMG']
        stats['CV'] = 5*2 + 50 + (62.2 if circlet == 'CRIT' else 0) + self.extra_stats['CV'] # 'Effective CV' as a plain sum of 2x CR and CD, assuming balanced stats
        # Ascension stats
        asc_stat_char = self.base_stats['asc_stat']
        asc_stat_weap = weapon.asc['stat']
        stat_mults = (self.base_stats['HP']/100, self.base_atk(weapon)/100, self.base_stats['DEF']/100, 1, 1, 1, 1)
        if asc_stat_char != 'None':
            stats[asc_stat_char] = stats[asc_stat_char] + self.base_stats['asc_value']*stat_mults[list(stats.keys()).index(asc_stat_char)]
        if asc_stat_weap != 'None':
            stats[asc_stat_weap] = stats[asc_stat_weap] + weapon.asc[weapon.level]*stat_mults[list(stats.keys()).index(asc_stat_weap)]
        return stats
    
    def apply_self_buffs(self, stats):
        # Independent self-buffs such as ascension passives giving flat stat increases that can be assumed to be always active
        return stats

    def apply_conversions(self, stats):
        # Post-buff conversions such as Hu Tao's PP and Itto's burst. Basic hypercarry doesn't have any self-buffs, so the values are just returned
        return stats

class Weapon_base():
    def __init__(self, holder=None, name='Basic weapon', atk_values=(380, 440, 500), asc_stat='None', asc_values=(0, 0, 0), psv_type='None', psv_range=(False,), psv_value=((None,),)):
        # Because a weapon holds relatively little information, I did not create a non-widget class for them
        # Values are given for levels 70/70, 80/80, 90/90 because otherwise it's too much numbers and likely irrelevant anyhow.
        self.holder = holder # Required to pass some base stats for weapon passives
        self.name = name
        self.level = '90/90'
        self.refine = 1
        self.atk = {'70/70': atk_values[0], '80/80': atk_values[1], '90/90': atk_values[2]}
        self.asc = {'stat': asc_stat, '70/70': asc_values[0], '80/80': asc_values[1], '90/90': asc_values[2]}
        self.psv = {'type': psv_type, 'range': psv_range, 'active': psv_range[0], 'value': psv_value}
    
    def calculate_refines(self):
        psv_value = [] # Handling refines additively
        for i in range(len(self.psv['value'][0])): psv_value.append(self.psv['value'][0][i] + self.psv['value'][1][i]*(self.refine - 1))
        return psv_value
    
    def apply_buffs(self, stats):
        # Apply weapon buffs to stats: basic weapon has none
        return stats

    def apply_conversions(self, stats):
        # Apply conversions to stats: basic weapon has none
        return stats

class Char_buffer(Char_placeholder):
    def __init__(self, name='None', element='Physical'):
        super(Char_buffer, self).__init__(name, element)
        # Abstract party member that provides elemental resonance, but no direct buffs
        self.fields = tuple()

    def buff(self, stats, hypercarry, weapon):
        return

In [5]:
# Hypercarries
class Char_hypercarry_custom(Char_hypercarry):
    def __init__(self):
        super(Char_hypercarry_custom, self).__init__('Custom hypercarry', 'Physical', ('HP', 'ATK', 'DEF', 'EM', 'ER', 'DMG', 'CV'))
        self.fields = (
            ['Custom hypercarry has no special optional properties', 'text', None, None],
        )

class Char_ayaka(Char_hypercarry):
    def __init__(self):
        super(Char_ayaka, self).__init__('Kamisato Ayaka', 'Cryo', ('ATK', 'DMG', 'CV'))
        # Default Ayaka is assumed to be lvl 90, holding a Amenoma Kageuchi and 4p Blizzard Strayer. C4 is currently not counted because it is finicky
        self.base_stats['HP'] = 12858
        self.base_stats['ATK'] = 342
        self.base_stats['DEF'] = 783
        self.base_stats['asc_stat'] = 'CV'
        self.base_stats['asc_value'] = 38.4
        self.weapon_types.update({
            'Amenoma Kageuchi': Widget_weapon(Weapon_amenoma_kageuchi, self),
            'Finale of the Deep': Widget_weapon(Weapon_finale_of_the_deep, self),
            'Mistsplitter Reforged': Widget_weapon(Weapon_mistsplitter_reforged, self)
        })
        self.weapon = self.weapon_types['Amenoma Kageuchi'].weapon
        self.fields = (
            ['Blizzard Strayer', 'str', ('No 4p bonus', '4p Cryo', '4p Freeze'), '4p Freeze'],
        )

    def apply_self_buffs(self, stats):
        # Ayaka benefits from A1 (negligible so doesn't count) and A4, can benefit from 4p BS
        stats['DMG'] = stats['DMG'] + 18
        stats['CV'] = stats['CV'] + (40 if self.fields[0][3] == '4p Cryo' else 0) + (80 if self.fields[0][3] == '4p Freeze' else 0)
        return stats
    
class Char_hutao(Char_hypercarry):
    def __init__(self):
        super(Char_hutao, self).__init__('Hu Tao', 'Pyro', ('HP', 'ATK', 'DMG', 'CV'))
        # Default Hu Tao is assumed to be lvl 90, holding a 500 ATK weapon and 4p Crimson Witch of Flames, healthy. Constellations are currently not counted because they are finicky
        self.base_stats['HP'] = 15552
        self.base_stats['ATK'] = 106
        self.base_stats['DEF'] = 876
        self.base_stats['asc_stat'] = 'HP'
        self.base_stats['asc_value'] = 38.4
        self.fields = (
            ['Talent level', 'int', (1, 13), 8],
            ['Under 50% health', 'bool', None, False],
            ['Artifact set bonus', 'str', ('No 4p set bonus', 'Crimson Witch of Flames', 'Shimenawa\'s Reminiscence'), 'Crimson Witch of Flames']
        )

    def apply_self_buffs(self, stats):
        # Hu Tao can benefit from A4, 4p CW and 4p SR
        stats['DMG'] = stats['DMG'] + (33 if self.fields[1][3] else 0) + (0, 15*0.5, 50)[self.fields[2][2].index(self.fields[2][3])]
        return stats

    def apply_conversions(self, stats):
        # Hu Tao converts max HP to ATK
        stats['ATK'] = stats['ATK'] + min(self.base_atk()*4, stats['HP']*((3.84, 4.07, 4.3, 4.6, 4.83, 5.06, 5.36, 5.66, 5.96, 6.26, 6.55, 6.85, 7.15)[self.fields[0][3]])/100)
        return stats
    
class Char_ei(Char_hypercarry):
    def __init__(self):
        super(Char_ei, self).__init__('Raiden Shogun', 'Electro', ('ATK', 'ER', 'DMG', 'CV'))
        # Default Ei is assumed to be lvl 90 holding The Catch R5 at lvl 90 and 4p Emblem of Severed Fate
        self.base_stats['HP'] = 12907
        self.base_stats['ATK'] = 337
        self.base_stats['DEF'] = 789
        self.base_stats['asc_stat'] = 'ER'
        self.base_stats['asc_value'] = 32
        self.weapon_types.update({
            'The Catch': Widget_weapon(Weapon_the_catch, self),
            'Engulfing Lightning': Widget_weapon(Weapon_engulfing_lightning, self)
        })
        self.weapon = self.weapon_types['The Catch'].weapon
        self.weapon_types['The Catch'].widget_psv.setChecked(True) # Since Ei uses burst in HC mode
        self.weapon_types['Engulfing Lightning'].widget_psv.setChecked(True)
        self.fields = (
            ['Holds 4p Emblem of Severed Fate', 'bool', None, True],
        )
    
    def apply_conversions(self, stats):
        # Ei converts ER to DMG% with her A4, can benefit from 4p EosF
        stats['DMG'] = stats['DMG'] + 0.4*(stats['ER'] - 100) + (max(0.25*stats['ER'], 75) if self.fields[0][3] else 0)
        return stats
       
class Char_diluc(Char_hypercarry):
    def __init__(self):
        super(Char_diluc, self).__init__('Diluc', 'Pyro', ('ATK', 'DMG', 'CV'))
        # Default Diluc is assumed to be lvl 90 holding a 500 ATK weapon and 4p Crimson Witch of Flames. Constellations are currently not counted because they are finicky
        self.base_stats['HP'] = 12980
        self.base_stats['ATK'] = 334
        self.base_stats['DEF'] = 783
        self.base_stats['asc_stat'] = 'CV'
        self.base_stats['asc_value'] = 38.4
        self.weapon_types.update({
            'Serpent Spine': Widget_weapon(Weapon_serpent_spine, self),
            'Redhorn Stonethresher': Widget_weapon(Weapon_redhorn_stonethresher, self)
        })
        self.weapon_types['Serpent Spine'].widget_psv.setValue(5) # We can assume it is maxed if used properly
        self.weapon_types['Redhorn Stonethresher'].widget_psv.setValue(60) # Eyeballing for Diluc
        self.fields = (
            ['Holds 4p Crimson Witch', 'bool', None, True],
        )
    
    def apply_self_buffs(self, stats):
        # Diluc benefits from A4, and can benefit from 4p CW
        stats['DMG'] = stats['DMG'] + 20 + (15*1.5 if self.fields[0][3] else 0)
        return stats

class Char_itto(Char_hypercarry):
    def __init__(self):
        super(Char_itto, self).__init__('Arataki Itto', 'Geo', ('ATK', 'DEF', 'DMG', 'CV'))
        # Default Itto is assumed to be lvl 90 holding a 500 ATK weapon and 4p Husk of Opulent Dreams. Constellations are currently not counted because they are finicky
        self.base_stats['HP'] = 12858
        self.base_stats['ATK'] = 227
        self.base_stats['DEF'] = 959
        self.base_stats['asc_stat'] = 'CV'
        self.base_stats['asc_value'] = 38.4
        self.weapon_types.update({
            'Serpent Spine': Widget_weapon(Weapon_serpent_spine, self),
            'Redhorn Stonethresher': Widget_weapon(Weapon_redhorn_stonethresher, self)
        })
        self.weapon_types['Serpent Spine'].widget_psv.setValue(5) # We can assume it is maxed if used properly
        self.weapon_types['Redhorn Stonethresher'].widget_psv.setValue(80) # Since Itto's damage mostly comes from normal/charged attacks
        self.fields = (
            ['Talent level', 'int', (1, 13), 8],
            ['Holds 4p Husk of Opulent Dreams', 'bool', None, True]
        )
    
    def apply_self_buffs(self, stats):
        # Itto benefits from A4 (a 35% extra DEF conversion for Kesagiri results in... 20% average?), and can benefit from 4p Husk, which he maxes out almost instantly
        stats['DEF'] = stats['DEF'] + (20 + (6*4 if self.fields[1][3] else 0))*self.base_stats['DEF']/100
        stats['DMG'] = stats['DMG'] + (6*4 if self.fields[1][3] else 0)
        return stats

    def apply_conversions(self, stats):
        # Itto converts DEF to ATK
        stats['ATK'] = stats['ATK'] + stats['DEF']*((57.6, 61.92, 66.24, 72, 76.32, 80.64, 86.4, 92.16, 97.92, 103.68, 109.44, 115.2, 122.4)[self.fields[0][3]])/100
        return stats

MODULE_hc_types = {
    'Basic hypercarry': Char_hypercarry,
    'Kamisato Ayaka': Char_ayaka,
    'Hu Tao': Char_hutao,
    'Raiden Shogun': Char_ei,
    'Diluc': Char_diluc,
    'Arataki Itto': Char_itto,
    'Custom hypercarry': Char_hypercarry_custom
}

In [6]:
# Weapons
class Weapon_custom(Weapon_base):
    def __init__(self, holder = None):
        super(Weapon_custom, self).__init__()
        # Custom weapon: customizable stats, no passive
        self.name = 'Custom weapon'
        self.level = 0
        self.atk = [500]
        self.asc = {'stat': 'None', 0: 0}

class Weapon_amenoma_kageuchi(Weapon_base):
    def __init__(self, holder = None):
        super(Weapon_amenoma_kageuchi, self).__init__(holder, 'Amenoma Kageuchi', (347, 401, 454), 'ATK', (45.4, 50.3, 55.1), 'Does not affect stats directly')

class Weapon_finale_of_the_deep(Weapon_base):
    def __init__(self, holder = None):
        super(Weapon_finale_of_the_deep, self).__init__(holder, 'Finale of the Deep', (429, 497, 565), 'ATK', (22.7, 25.1, 27.6), 'stacks', (0, 2), ((12, 150), (3, 37.5)))
        
    def apply_buffs(self, stats):
        psv_value = self.calculate_refines()
        # Finale of the Deep increases ATK by psv_value[0] after using a skill and further by psv_value[1] if the lifebond is cleared
        stats['ATK'] = stats['ATK'] + (psv_value[0] if self.psv['active'] >= 1 else 0)*self.holder.base_atk(self)/100
        stats['ATK'] = stats['ATK'] + (psv_value[1] if self.psv['active'] >= 2 else 0)
        return stats

class Weapon_mistsplitter_reforged(Weapon_base):
    def __init__(self, holder = None):
        super(Weapon_mistsplitter_reforged, self).__init__(holder, 'Mistsplitter Reforged', (506, 590, 674), 'CV', (36.3, 40.2, 44.1), 'stacks', (0, 3), ((12, 8), (3, 2)))
        
    def apply_buffs(self, stats):
        psv_value = self.calculate_refines()
        # Mistsplitter Reforged provides an unconditional psv_value[0] DMG% bonus and 1x/2x/3.5x psv_value[1] DMG% bonus from stacks
        stats['DMG'] = stats['DMG'] + psv_value[0] + psv_value[1]*(0, 1, 2, 3.5)[self.psv['active']]
        return stats
    
class Weapon_the_catch(Weapon_base):
    def __init__(self, holder = None):
        super(Weapon_the_catch, self).__init__(holder, 'The Catch', (388, 449, 510), 'ER', (37.9, 41.9, 45.9), 'bool', (False, True), ((16, 6), (4, 1.5)))
        # The Catch is assumed to be R5 because why not
        self.refine = 5
        
    def apply_buffs(self, stats):
        psv_value = self.calculate_refines()
        # The Catch increases DMG by psv_value[0] and CRIT Rate by psv_value[1] if what we are calculating for is an elemental burst
        stats['DMG'] = stats['DMG'] + (psv_value[0] if self.psv['active'] == True else 0)
        stats['CV'] = stats['CV'] + (psv_value[1]*2 if self.psv['active'] == True else 0)
        return stats
        
class Weapon_engulfing_lightning(Weapon_base):
    def __init__(self, holder = None):
        super(Weapon_engulfing_lightning, self).__init__(holder, 'Engulfing Lightning', (457, 532, 608), 'ER', (45.4, 50.3, 55.1), 'bool', (False, True), ((28, 80), (7, 10)))

    def apply_buffs(self, stats):
        # Engulfing Lightning increases ER by 30% if what we are calculating for is an elemental burst
        stats['ER'] = stats['ER'] + (30 if self.psv['active'] == True else 0)
        return stats

    def apply_conversions(self, stats):
        psv_value = self.calculate_refines()
        # Engulfing Lightning converts ER over 100 to ATK at psv_value[0]*base_atk efficiency, up to psv_value[1]% total
        stats['ATK'] = stats['ATK'] + max(psv_value[1], psv_value[0]*(stats['ER']/100 - 1))*self.holder.base_atk(self)/100
        return stats
    
class Weapon_serpent_spine(Weapon_base):
    def __init__(self, holder = None):
        super(Weapon_serpent_spine, self).__init__(holder, 'Serpent Spine', (388, 449, 510), 'CV', (45.4, 50.3, 55.1), 'stacks', (0, 5), ((6,), (1,)))
        
    def apply_buffs(self, stats):
        psv_value = self.calculate_refines()
        # Serpent Spine provides psv_value[0] DMG% bonus for every stack
        stats['DMG'] = stats['DMG'] + psv_value[0]*self.psv['active']
        return stats

class Weapon_redhorn_stonethresher(Weapon_base):
    def __init__(self, holder = None):
        super(Weapon_redhorn_stonethresher, self).__init__(holder, 'Redhorn Stonethresher', (408, 475, 542), 'CV', (72.7, 80.4, 88.2), 'perc', (0, 100), ((28, 40), (7, 10)))
        
    def apply_buffs(self, stats):
        psv_value = self.calculate_refines()
        # Redhorn Stonethresher provides psv_value[0] DEF% bonus and increases Normal/Charged attack DMG% by psv_value[1]
        stats['DEF'] = stats['DEF'] + psv_value[0]*self.holder.base_stats['DEF']/100
        stats['DMG'] = stats['DMG'] + (psv_value[1] if self.psv['active'] else 0)
        return stats        

In [7]:
# Buffers
class Char_vv(Char_buffer):
    def __init__(self):
        super(Char_vv, self).__init__('VV provider', 'Anemo')
        # Somebody doing a VV who is not Kazuha
        # Faruzan can technically hold VV, but she only buffs Anemo damage which can't be VVed so she is a separate case entirely
        self.fields = (
            ['Providing elemental RES shred through 4p Viridescent Venerer swirls', 'text', None, None],
        )

    def is_relevant(self, hypercarry):
        return hypercarry.element in ('Pyro', 'Hydro', 'Electro', 'Cryo')
    
    def buff(self, stats, hypercarry, weapon):
        if hypercarry.element in ('Pyro', 'Hydro', 'Electro', 'Cryo'): stats['shred_vv'] = 40
    
class Char_bennett(Char_buffer):
    def __init__(self):
        super(Char_bennett, self).__init__('Bennett', 'Pyro')
        # Default Bennett is assumed to have Base ATK of 750 (approx. lvl 90 Sapwood 90), talent level of 8, C0, and has 4p Noblesse Obligue
        self.fields = (
            ['Base ATK', 'int', (0, 9999), 750],
            ['Talent level', 'int', (1, 13), 8],
            ['Constellation', 'int', (0, 6), 0],
            ['Holds 4p Noblesse Obligue', 'bool', None, True]
        )
    
    def is_relevant(self, hypercarry):
        return 'ATK' in hypercarry.scales_with or ('DMG' in hypercarry.scales_with and hypercarry.element == 'Pyro')
        
    def buff_value_atk(self):
        return self.fields[0][3]*((56, 60.2, 64.4, 70, 74.2, 78.4, 84, 89.6, 95.2, 100.8, 106.4, 112, 119, 126)[self.fields[1][3]-1] + (20 if self.fields[2][3] >= 1 else 0))/100
    
    def buff(self, stats, hypercarry, weapon):
        stats['ATK'] = stats['ATK'] + self.buff_value_atk() + (0.2*hypercarry.base_atk(weapon) if self.fields[3][3] else 0)
        stats['DMG'] = stats['DMG'] + (15 if self.fields[2][3] == 6 and hypercarry.element == 'Pyro' else 0)
     
class Char_sara(Char_buffer):
    def __init__(self):
        super(Char_sara, self).__init__('Sara', 'Electro')
        # Default Sara is assumed to have Base ATK of 750 (approx. lvl 90 565-bow 90), talent level of 8 and no C6
        self.fields = (
            ['Base ATK', 'int', (0, 9999), 750],
            ['Talent level', 'int', (1, 13), 8],
            ['Is C6', 'bool', None, False]
        )
        
    def is_relevant(self, hypercarry):
        return 'ATK' in hypercarry.scales_with or ('CV' in hypercarry.scales_with and hypercarry.element == 'Electro')
        
    def buff_value_atk(self): # CDMG buff is added separately, that's why we need carry's crit stats
        return self.fields[0][3]*((43, 46, 49, 54, 57, 60, 64.4, 69, 73.0, 77.3, 81.6, 85.9, 91.2, 97)[self.fields[1][3]-1])/100

    def buff(self, stats, hypercarry, weapon):
        stats['ATK'] = stats['ATK'] + self.buff_value_atk()
        stats['CV'] = stats['CV'] + (60 if self.fields[2][3] == True and hypercarry.element == 'Electro' else 0)

class Char_furina(Char_buffer):
    def __init__(self):
        super(Char_furina, self).__init__('Furina', 'Hydro')
        # Default Furina is assumed to have average Fanfare charge percent of 75, talent level of 8 and no C1
        self.fields = (
            ['Avg. Fanfare', 'int', (0, 300), 200],
            ['Talent level', 'int', (1, 13), 8],
            ['Is C1', 'bool', None, False]
        )

    def is_relevant(self, hypercarry):
        return 'DMG' in hypercarry.scales_with
        
    def buff_value_dmg(self): # This is a DMG% buff, not ATK
        return (0.07, 0.09, 0.11, 0.13, 0.15, 0.17, 0.19, 0.21, 0.23, 0.25, 0.27, 0.29, 0.31)[self.fields[1][3]-1]*((150+max(self.fields[0][3], 250)) if self.fields[2][3] else (self.fields[0][3]))
       
    def buff(self, stats, hypercarry, weapon):
        stats['DMG'] = stats['DMG'] + self.buff_value_dmg()

class Char_kazuha(Char_buffer):
    def __init__(self):
        super(Char_kazuha, self).__init__('Kaedehara Kazuha', 'Anemo')
        # Default Kazuha is expected to have 800 EM and no C2
        self.fields = (
            ['Elemental mastery', 'int', (0, 1999), 800],
            ['Is C2', 'bool', None, False],
            ['Holds 4p Viridescent Venerer', 'bool', None, True]
        )

    def is_relevant(self, hypercarry):
        return 'EM' in hypercarry.scales_with or hypercarry.element in ('Pyro', 'Hydro', 'Electro', 'Cryo')
        
    def buff_value_dmg(self, stats):
        return 0.04*(self.fields[0][3] + (200 if self.fields[1][3] else 0) + (75 if stats['elements_in_party'].count('Dendro') >= 2 else 0))

    def buff(self, stats, hypercarry, weapon):
        if self.fields[1][3]: stats['EM'] = stats['EM'] + 200
        if hypercarry.element in ('Pyro', 'Hydro', 'Electro', 'Cryo'):
            stats['DMG'] = stats['DMG'] + self.buff_value_dmg(stats)
            if self.fields[2][3]: stats['shred_vv'] = 40
    
class Char_chevreuse(Char_buffer):
    def __init__(self):
        super(Char_chevreuse, self).__init__('Chevreuse', 'Pyro')
        # Default Chevreuse is assumed to have 35k hp, and have 0 C6 stacks active
        self.fields = (
            ['HP', 'int', (1, 40000), 35000],
            ['C6 stacks', 'int', (0, 3), 0]
        )
        
    def is_relevant(self, hypercarry):
        return hypercarry.element in ('Pyro', 'Electro')
        
    def buff_value_dmg(self):
        return 0.001*self.fields[0][3] + 20*self.fields[1][3]
    
    def buff(self, stats, hypercarry, weapon):
        if set(stats['elements_in_party']) == set(('Pyro', 'Electro')):
            stats['DMG'] = stats['DMG'] + self.buff_value_dmg() # !! UNFINISHED !! Chevreuse instance cannot know if there is a Hydro resonance
            stats['shred_chevreuse'] = 40

class Char_zhongli(Char_buffer):
    def __init__(self):
        super(Char_zhongli, self).__init__('Zhongli', 'Geo')
        # Default Zhongli provides his shield and holds 4p Tenacity of the Millelith
        self.fields = (
            ['Holds 4p Tenacity of the Millelith', 'bool', None, True],
        )
  
    def is_relevant(self, hypercarry):
        return True
        
    def buff(self, stats, hypercarry, weapon):
        stats['ATK'] = stats['ATK'] + (0.2*hypercarry.base_atk(weapon) if self.fields[0][3] else 0)
        stats['shred_zhongli'] = 20
    
class Char_gorou(Char_buffer):
    def __init__(self):
        super(Char_gorou, self).__init__('Gorou', 'Geo')
        # Default Gorou is assumed to have talent level of 8, no C6, and no 4p Noblesse Obligue
        self.fields = (
            ['Talent level', 'int', (1, 13), 8],
            ['Is C6', 'bool', None, False],
            ['Holds 4p Noblesse Obligue', 'bool', None, True]
        )
    
    def is_relevant(self, hypercarry):
        return 'DEF' in hypercarry.scales_with or (('DMG' or 'CV') in hypercarry.scales_with and hypercarry.element == 'Geo')
        
    def buff_value_def(self):
        return (206.16, 221.62, 237.08, 257.7, 273.16, 288.62, 309.24, 329.85, 350.47, 371.08, 391.70, 412.32, 438.09)[self.fields[0][3]-1]
    
    def buff(self, stats, hypercarry, weapon):
        stats['ATK'] = stats['ATK'] + (0.2*hypercarry.base_atk(weapon) if self.fields[2][3] else 0)
        stats['DEF'] = stats['DEF'] + self.buff_value_def() + 0.25*hypercarry.base_stats['DEF']
        stats['DMG'] = stats['DMG'] + (15 if stats['elements_in_party'].count('Geo') >= 3 and hypercarry.element == 'Geo' else 0)
        stats['CV'] = stats['CV'] + (40 if self.fields[1][3] == True and hypercarry.element == 'Geo' else 0)

MODULE_buffer_types = {
    'None': Char_placeholder,
    'Bennett': Char_bennett,
    'Kujou Sara': Char_sara,
    'Furina': Char_furina,
    'Kaedehara Kazuha': Char_kazuha,
    'Chevreuse': Char_chevreuse,
    'Zhongli': Char_zhongli,
    'Gorou': Char_gorou,
    'VV provider': Char_vv,
    'Resonance provider': Char_placeholder
}

In [8]:
# Widgets - adaptive
class Widget_hypercarry(Widget_char_placeholder):
    def __init__(self, char_type = Char_hypercarry):
        super(Widget_hypercarry, self).__init__(char_type)
        layout_main = QtWidgets.QVBoxLayout(self)
        # Base stats
        label_stats_base = QtWidgets.QLabel('Base stats:', self)
        layout_stats_base = QtWidgets.QHBoxLayout()
        if char_type != Char_hypercarry_custom: # Static stats for existing characters
            self.widgets_stats_base = []
            self.widgets_stats_base.append(make_labled_box(self, 'Element', 'bold', None, self.character.element))
            for attr in ('HP', 'ATK', 'DEF'):
                if attr in self.character.scales_with: self.widgets_stats_base.append(make_labled_box(self, attr, 'bold', None, str(self.character.base_stats[attr])))
            self.widgets_stats_base.append(make_labled_box(self, 'Ascension stat', 'bold', None, self.character.base_stats['asc_stat'] + ('' if self.character.base_stats['asc_stat'] == 'None' else '  -')))
            self.widgets_stats_base.append(make_labled_box(self, None, 'bold', None, ('' if self.character.base_stats['asc_stat'] == 'None'
                                                      else str(self.character.base_stats['asc_value']) + ('%' if self.character.base_stats['asc_stat'] != 'CV' else '')) ))
            for item in self.widgets_stats_base:
                if item['label'] != None: layout_stats_base.addWidget(item['label'])
                if item['box'] != None: layout_stats_base.addWidget(item['box'])
        else: # Editable stats for abstract hypercarry
            self.widgets_stats_base = []
            self.stats_base_types = (
                ['Element', 'str', ('Physical', 'Anemo', 'Pyro', 'Hydro', 'Electro', 'Cryo', 'Geo', 'Dendro'), self.character.element],
                ['HP', 'int', (1, 50000), self.character.base_stats['HP']],
                ['ATK', 'int', (1, 1000), self.character.base_stats['ATK']],
                ['DEF', 'int', (1, 2000), self.character.base_stats['DEF']],
                ['Ascension stat', 'str', ['None'] + list(self.character.scales_with), self.character.base_stats['asc_stat']],
                ['-', 'double', (0, 1000), self.character.base_stats['asc_value']]
            )
            for item in self.stats_base_types: self.widgets_stats_base.append(make_labled_box(self, *item, self.update_char_base))
            for item in self.widgets_stats_base:
                if item['label'] != None: layout_stats_base.addWidget(item['label'])
                if item['box'] != None: layout_stats_base.addWidget(item['box'])
        layout_stats_base.addSpacerItem(QtWidgets.QSpacerItem(0, 0, QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Minimum))
        # Weapon
        layout_label_weapon = QtWidgets.QHBoxLayout()
        label_weapon = QtWidgets.QLabel('Weapon:', self)
        self.box_weapon = make_combo_box(self, tuple(self.character.weapon_types.keys()), self.character.weapon.name, self.update_weapon)
        layout_label_weapon.addWidget(label_weapon)
        layout_label_weapon.addWidget(self.box_weapon)
        layout_label_weapon.addSpacerItem(QtWidgets.QSpacerItem(0, 0, QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Minimum))
        self.layout_weapon = QtWidgets.QStackedLayout()
        for item in self.character.weapon_types.values(): self.layout_weapon.addWidget(item)
        self.layout_weapon.setCurrentIndex(tuple(self.character.weapon_types.keys()).index(self.character.weapon.name))
        frame_weapon = make_basic_frame(self.layout_weapon)
        # Artifacts
        label_artifacts = QtWidgets.QLabel('Artifacts: types by main stat', self)
        layout_artifacts = QtWidgets.QHBoxLayout()
        widgets_to_add = []
        self.boxes_artifacts = {}
        for slot in ('Sands', 'Goblet', 'Circlet'):
            widgets_to_add.append(make_labled_box(self, slot, 'str', self.character.artifact_types[slot], self.character.artifacts[slot], self.update_equipment))
            self.boxes_artifacts.update({slot: widgets_to_add[-1]['box']})
        for item in widgets_to_add:
            layout_artifacts.addWidget(item['label'])
            layout_artifacts.addWidget(item['box'])            
        layout_artifacts.addSpacerItem(QtWidgets.QSpacerItem(0, 0, QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Minimum))
        # Extra stats
        label_stats_extra = QtWidgets.QLabel('Extra stats: artifact substats, set bonuses', self)
        layout_stats_extra = QtWidgets.QHBoxLayout()
        widgets_to_add = []
        self.boxes_extra = {}
        for stat in self.character.scales_with:
            if stat in ('HP', 'ATK', 'DEF'): # Cannot be really optimized with make_labled_box because of the grid layouts, I can save a couple lines at best
                layout_stat = QtWidgets.QGridLayout()
                label_stat_flat = QtWidgets.QLabel('Flat ' + stat + ':', self)
                self.boxes_extra[stat + '_flat'] = make_spin_box(self, (0, 9999), self.character.extra_stats[stat + '_flat'], self.update_equipment)
                label_stat_perc = QtWidgets.QLabel(stat + '%:', self)
                self.boxes_extra[stat + '_perc'] = make_double_spin_box(self, (0, 999), self.character.extra_stats[stat + '_perc'], self.update_equipment)
                layout_stat.addWidget(label_stat_flat, 0, 0)
                layout_stat.addWidget(label_stat_perc, 1, 0)
                layout_stat.addWidget(self.boxes_extra[stat + '_flat'], 0, 1)
                layout_stat.addWidget(self.boxes_extra[stat + '_perc'], 1, 1)
                widgets_to_add = widgets_to_add + [layout_stat]
                del layout_stat # Clearing references, the elements themselves are already attached to something and as such won't be deleted
                del label_stat_flat
                del label_stat_perc
            elif stat == 'EM': # EM is an integer unlke all other single-number stats
                label_stat = QtWidgets.QLabel(stat + ':', self)
                self.boxes_extra[stat] = make_spin_box(self, (0, 999), self.character.extra_stats[stat], self.update_equipment)
                widgets_to_add = widgets_to_add + [label_stat, self.boxes_extra[stat]]
                del label_stat
            else:
                label_stat = QtWidgets.QLabel(stat + ':', self)
                self.boxes_extra[stat] = make_double_spin_box(self, (0, 999), self.character.extra_stats[stat], self.update_equipment)
                widgets_to_add = widgets_to_add + [label_stat, self.boxes_extra[stat]]
                del label_stat
        for item in widgets_to_add:
            if type(item) in (QtWidgets.QHBoxLayout, QtWidgets.QVBoxLayout, QtWidgets.QGridLayout, QtWidgets.QStackedLayout): layout_stats_extra.addLayout(item)
            else: layout_stats_extra.addWidget(item)
        layout_stats_extra.addSpacerItem(QtWidgets.QSpacerItem(0, 0, QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Minimum))
        # Character-specific fields
        layout_char_fields = QtWidgets.QHBoxLayout()
        self.widgets_fields = []
        for item in self.character.fields:
            self.widgets_fields.append(make_labled_box(self, *item, self.update_char_fields))
        for item in self.widgets_fields: 
            layout_char_fields.addWidget(item['label'])
            layout_char_fields.addWidget(item['box'])
        layout_char_fields.addSpacerItem(QtWidgets.QSpacerItem(0, 0, QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Minimum))
        #
        for item in (label_stats_base, layout_stats_base, layout_label_weapon, frame_weapon, label_artifacts, layout_artifacts, label_stats_extra, layout_stats_extra, layout_char_fields):
            if type(item) in (QtWidgets.QHBoxLayout, QtWidgets.QVBoxLayout, QtWidgets.QGridLayout, QtWidgets.QStackedLayout): layout_main.addLayout(item)
            else: layout_main.addWidget(item)
        layout_main.addSpacerItem(QtWidgets.QSpacerItem(0, 0, QtWidgets.QSizePolicy.Minimum, QtWidgets.QSizePolicy.Expanding))

    def update_char_base(self): # This function refers to attributes that only exist for abstract hypercarry, but should only be called by an abstract hypercarry
        self.character.element = self.widgets_stats_base[0]['box'].currentText()
        if 'HP' in self.character.scales_with: self.character.base_stats['HP'] = self.widgets_stats_base[1]['box'].value()
        if 'ATK' in self.character.scales_with: self.character.base_stats['ATK'] = self.widgets_stats_base[2]['box'].value()
        if 'DEF' in self.character.scales_with: self.character.base_stats['DEF'] = self.widgets_stats_base[3]['box'].value()
        self.character.base_stats['asc_stat'] = self.widgets_stats_base[4]['box'].currentText()
        self.character.base_stats['asc_value'] = self.widgets_stats_base[5]['box'].value()

    def update_weapon(self):
        weapon_name = self.box_weapon.currentText()
        if weapon_name != self.character.weapon.name:
            self.layout_weapon.setCurrentIndex(tuple(self.character.weapon_types.keys()).index(weapon_name))
            self.character.weapon = self.layout_weapon.currentWidget().weapon # Theoretically, any calculation character does with the weapon can be handled by character object asking it's parent widget
        
    def update_equipment(self):
        for stat in self.character.scales_with:
            if stat in ('HP', 'ATK', 'DEF'):
                self.character.extra_stats[stat + '_flat'] = self.boxes_extra[stat + '_flat'].value()
                self.character.extra_stats[stat + '_perc'] = self.boxes_extra[stat + '_perc'].value()
            else:
                self.character.extra_stats[stat] = self.boxes_extra[stat].value()
        for slot in ('Sands', 'Goblet', 'Circlet'):
            self.character.artifacts[slot] = self.boxes_artifacts[slot].currentText()

    def update_char_fields(self):
        for item, widget in zip(self.character.fields, self.widgets_fields):
            item[3] = getattr(widget['box'], {'bool': 'isChecked', 'str': 'currentText', 'int': 'value', 'double': 'value'}[item[1]])()

class Widget_weapon(QtWidgets.QWidget):
    def __init__(self, weapon_type, holder):
        super(Widget_weapon, self).__init__()
        self.weapon = weapon_type(holder=holder)
        self.custom_weapon = (weapon_type == Weapon_custom)
        #
        layout_main = QtWidgets.QVBoxLayout(self)
        layout_1 = QtWidgets.QHBoxLayout()
        if self.custom_weapon:
            fields = (
                ['ATK', 'int', (1, 1999), self.weapon.atk[0]],
                ['Ascension stat', 'str', ['None'] + list(holder.scales_with), self.weapon.asc['stat']],
                ['-', 'double', (0, 999), self.weapon.asc[self.weapon.level]],
                [('' if self.weapon.asc['stat'] == 'None' else str(self.weapon.asc[self.level]) + ('%' if self.weapon.asc['stat'] not in ('EM', 'CV') else '')), 'text', None, None]
            )
        else:
            fields = (
                ['Level', 'str', ('70/70', '80/80', '90/90'), self.weapon.level],
                ['Refinement', 'int', (1, 5), self.weapon.refine],
                ['ATK', 'bold', None, str(self.weapon.atk[self.weapon.level])],
                ['Ascension stat', 'bold', None, self.weapon.asc['stat']],
                ['-', 'bold', None, ('' if self.weapon.asc['stat'] == 'None' else str(self.weapon.asc[self.weapon.level]) + ('%' if self.weapon.asc['stat'] not in ('EM', 'CV') else ''))]
            )
        self.widgets_stats_base = []
        for item in fields:
            self.widgets_stats_base.append(make_labled_box(self, *item, self.update_weapon))
        for item in self.widgets_stats_base:
                if item['label'] != None: layout_1.addWidget(item['label'])
                if item['box'] != None: layout_1.addWidget(item['box'])
        layout_1.addSpacerItem(QtWidgets.QSpacerItem(0, 0, QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Minimum))
        #
        layout_2 = QtWidgets.QHBoxLayout()
        label_psv = QtWidgets.QLabel('Passive:', self)
        widgets_to_add = [label_psv]
        if self.weapon.psv['type'] == 'bool':
            self.widget_psv = make_check_box(self, '', self.weapon.psv['active'], self.update_weapon)
            widgets_to_add = widgets_to_add + [self.widget_psv]
        elif self.weapon.psv['type'] == 'stacks':
            self.widget_psv = make_spin_box(self, (self.weapon.psv['range'][0], self.weapon.psv['range'][1]), self.weapon.psv['active'], self.update_weapon)
            label_passive_extra = QtWidgets.QLabel('stacks', self)
            widgets_to_add = widgets_to_add + [self.widget_psv, label_passive_extra]
        elif self.weapon.psv['type'] == 'perc':
            self.widget_psv = make_double_spin_box(self, (self.weapon.psv['range'][0], self.weapon.psv['range'][1]), self.weapon.psv['active'], self.update_weapon)
            label_passive_extra = QtWidgets.QLabel('% effective', self)
            widgets_to_add = widgets_to_add + [self.widget_psv, label_passive_extra]
        else:
            label_passive_extra = QtWidgets.QLabel(self.weapon.psv['type'], self)
            widgets_to_add = widgets_to_add + [label_passive_extra]
        for item in widgets_to_add: layout_2.addWidget(item)
        layout_2.addSpacerItem(QtWidgets.QSpacerItem(0, 0, QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Minimum))
        #
        layout_main.addLayout(layout_1)
        layout_main.addLayout(layout_2)
        layout_main.addSpacerItem(QtWidgets.QSpacerItem(0, 0, QtWidgets.QSizePolicy.Minimum, QtWidgets.QSizePolicy.Expanding))
        
    def update_weapon(self):
        if self.custom_weapon:
            self.weapon.atk[self.weapon.level] = self.widgets_stats_base[0]['box'].value()
            self.weapon.asc['stat'] = self.widgets_stats_base[1]['box'].currentText()
            self.weapon.asc[self.weapon.level] = self.widgets_stats_base[2]['box'].value()
        else:
            self.weapon.level = self.widgets_stats_base[0]['box'].currentText()
            self.weapon.refine = self.widgets_stats_base[1]['box'].value()
            self.widgets_stats_base[2]['box'].setText(str(self.weapon.atk[self.weapon.level]))
            self.widgets_stats_base[3]['box'].setText(('' if self.weapon.asc['stat'] == 'None' else str(self.weapon.asc[self.weapon.level]) + ('%' if self.weapon.asc['stat'] not in ('EM', 'CV') else '')))
            if self.weapon.psv['type'] == 'bool':
                self.weapon.psv['active'] = self.widget_psv.isChecked()
            elif self.weapon.psv['type'] == 'stacks':
                self.weapon.psv['active'] = self.widget_psv.value()
            elif self.weapon.psv['type'] == 'perc':
                self.weapon.psv['active'] = self.widget_psv.value()
            else:
                return            

class Widget_buffer(Widget_char_placeholder):
    def __init__(self, char_type=Char_buffer):
        super(Widget_buffer, self).__init__(char_type)
        layout = QtWidgets.QHBoxLayout(self)
        self.widgets_fields = []
        for item in self.character.fields:
            self.widgets_fields.append(make_labled_box(self, *item, self.update_fields))
        for item in self.widgets_fields: 
            if item['label'] != None: layout.addWidget(item['label'])
            if item['box'] != None: layout.addWidget(item['box'])
        layout.addSpacerItem(QtWidgets.QSpacerItem(0, 0, QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Minimum))
            
    def update_fields(self):
        for item, widget in zip(self.character.fields, self.widgets_fields):
            if item[1] not in ['text', 'bold']: item[3] = getattr(widget['box'], {'bool': 'isChecked', 'str': 'currentText', 'int': 'value', 'double': 'value'}[item[1]])()

class Widget_resonator(Widget_char_placeholder):
    def __init__(self, element = 'Physical'):
        super(Widget_resonator, self).__init__(Char_buffer)
        self.character.name = 'Resonator: ' + element
        #
        label_element = QtWidgets.QLabel('Element:', self)
        self.box_element = make_combo_box(self, ('Physical', 'Anemo', 'Pyro', 'Hydro', 'Electro', 'Cryo', 'Geo', 'Dendro'), self.character.element, self.update_char)
        #
        layout = QtWidgets.QHBoxLayout(self)
        for item in (label_element, self.box_element): layout.addWidget(item)
        layout.addSpacerItem(QtWidgets.QSpacerItem(0, 0, QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Minimum))
        
    def update_char(self):
        self.character.element = self.box_element.currentText()
        self.character.name = 'Resonator: ' + self.character.element

In [9]:
# Main UI widget
class Widget_genshin_buff_opt_main(QtWidgets.QWidget):
    def __init__(self):
        super(Widget_genshin_buff_opt_main, self).__init__()
        # Alternative, no-preload way to pick enabled hypercarries and buffers - by names
        self.hc_widgets = dict()
        for item in MODULE_hc_types.keys(): self.hc_widgets[item] = Widget_char_placeholder()
        # These arcane runes mean 'get first key and value from a dictionary' because even ordered since 3.7, they don't support indexing
        self.hc_widgets[tuple(MODULE_hc_types.keys())[0]] = Widget_hypercarry(tuple(MODULE_hc_types.values())[0])
        self.buffer_widgets = dict()
        for item in MODULE_buffer_types.keys(): self.buffer_widgets[item] = Widget_char_placeholder()
        buffer_names = tuple(MODULE_buffer_types.keys())
        self.placeholder_buffer_widgets = (Widget_char_placeholder(), Widget_char_placeholder(), Widget_char_placeholder()) # These are for the fields, not for preloads
        self.enemy_res = 10
        #
        layout_main = QtWidgets.QVBoxLayout(self)
        # Hypercarry
        layout_hc_s = QtWidgets.QHBoxLayout()
        label_hc = QtWidgets.QLabel('Hypercarry:', self)
        self.box_hc = make_combo_box(self, tuple(self.hc_widgets.keys()), tuple(MODULE_hc_types.keys())[0], self.switch_hypercarry)
        label_res = QtWidgets.QLabel('Enemy elemental resistance:', self)
        self.box_res = make_spin_box(self, (-100, 999), self.enemy_res, self.update_enemy_res)
        for item in (label_hc, self.box_hc, label_res, self.box_res): layout_hc_s.addWidget(item)
        layout_hc_s.addSpacerItem(QtWidgets.QSpacerItem(0, 0, QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Minimum))
        self.layout_hc = QtWidgets.QStackedLayout()
        for item in self.hc_widgets.values(): self.layout_hc.addWidget(item) # Populate with the default and placeholder widgets to avoid preloading hypercarries
        frame_hc = make_basic_frame(self.layout_hc)
        # Buffers
        self.layouts_buffers = (QtWidgets.QVBoxLayout(), QtWidgets.QVBoxLayout(), QtWidgets.QVBoxLayout()) # I need to keep those named to be able to reference them to switch out buffers
        self.boxes_buffers = (QtWidgets.QComboBox(self), QtWidgets.QComboBox(self), QtWidgets.QComboBox(self))
        for i in range(3):
            layout_buffer = QtWidgets.QHBoxLayout()
            label_buffer = QtWidgets.QLabel('Buffer ' + str(i+1) + ':', self)
            self.boxes_buffers[i].addItems(buffer_names)
            self.boxes_buffers[i].activated.connect(self.switch_buffers)
            layout_buffer.addWidget(label_buffer)
            layout_buffer.addWidget(self.boxes_buffers[i])
            layout_buffer.addSpacerItem(QtWidgets.QSpacerItem(0, 0, QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Minimum))
            self.layouts_buffers[i].addLayout(layout_buffer)
            self.layouts_buffers[i].addWidget(self.placeholder_buffer_widgets[i])
            self.layouts_buffers[i].addSpacerItem(QtWidgets.QSpacerItem(0, 0, QtWidgets.QSizePolicy.Minimum, QtWidgets.QSizePolicy.Expanding))
            del label_buffer
            del layout_buffer
        # Buttons
        layout_buttons = QtWidgets.QVBoxLayout(self)
        button_calculate = QtWidgets.QPushButton('Calculate damage output for the current setup', self)
        button_calculate.clicked.connect(self.calculate_setup)
        self.label_calc_result = QtWidgets.QLabel('Current configuration output is:', self)
        self.label_stats_result = QtWidgets.QLabel('Effective stats:', self)
        layout_buttons.addWidget(button_calculate)
        layout_buttons.addWidget(self.label_calc_result)
        layout_buttons.addWidget(self.label_stats_result)
        # Optimizers
        layout_opt_s = QtWidgets.QHBoxLayout()
        self.button_opt_build = QtWidgets.QPushButton('Build', self)
        self.button_opt_build.clicked.connect(self.switch_optimizer)
        self.button_opt_buffers = QtWidgets.QPushButton('Buffers', self)
        self.button_opt_buffers.clicked.connect(self.switch_optimizer)
        layout_opt_s.addWidget(self.button_opt_build)
        layout_opt_s.addWidget(self.button_opt_buffers)
        layout_opt_s.addSpacerItem(QtWidgets.QSpacerItem(0, 0, QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Minimum))
        widget_opt_build = Widget_optimize_build(self.box_res, self.layout_hc, self.layouts_buffers)
        widget_opt_buffers = Widget_optimize_buffers(self.box_res, self.layout_hc, self.get_buffer_widget)
        self.layout_optimizers = QtWidgets.QStackedLayout()
        for item in (widget_opt_build, widget_opt_buffers): self.layout_optimizers.addWidget(item)
        frame_opt = make_basic_frame(self.layout_optimizers)
        #
        for item in (layout_hc_s, frame_hc, self.layouts_buffers[0], self.layouts_buffers[1], self.layouts_buffers[2], layout_buttons, layout_opt_s, frame_opt):
            if type(item) in (QtWidgets.QHBoxLayout, QtWidgets.QVBoxLayout, QtWidgets.QGridLayout, QtWidgets.QStackedLayout): layout_main.addLayout(item)
            else: layout_main.addWidget(item)
        layout_main.addSpacerItem(QtWidgets.QSpacerItem(750, 0, QtWidgets.QSizePolicy.Minimum, QtWidgets.QSizePolicy.Expanding))
        
    # Note: 'hypercarry' and 'buffer' may be used interchangeably as name strings or character objects in different functions, pay attention
        
    def update_enemy_res(self):
        self.enemy_res = float(self.box_res.text())
    
    def get_hc_widget(self, hc_name):
        # Check if this hypercarry widget has already been made, make it and insert it into the hypercarry QStackedLayout if not, return it
        if self.hc_widgets[hc_name].character.name == 'None':
            old_w = self.hc_widgets[hc_name]
            self.hc_widgets[hc_name] = Widget_hypercarry(MODULE_hc_types[hc_name])
            self.layout_hc.replaceWidget(old_w, self.hc_widgets[hc_name])
            old_w.deleteLater()
            old_w = None
        return self.hc_widgets[hc_name]

    def switch_hypercarry(self):
        hc_name = self.box_hc.currentText()
        if hc_name != self.layout_hc.currentWidget().character.name: # Checking that the user actually picked someone new
            self.get_hc_widget(hc_name)
            self.layout_hc.setCurrentIndex(tuple(self.hc_widgets.keys()).index(hc_name))
    
    def get_buffer_widget(self, buffer_name):
        # If the buffer in question is a resonator, just make a new one
        if 'Resonator:' in buffer_name: return Widget_resonator(buffer_name.replace('Resonator: ', ''))
        # Check if this buffer widget has already been made, make it if not, return it if yes
        if self.buffer_widgets[buffer_name].character.name == 'None':
            self.buffer_widgets[buffer_name] = Widget_buffer(MODULE_buffer_types[buffer_name])
        return self.buffer_widgets[buffer_name]
        
    def switch_buffers(self):
        num_changed = self.boxes_buffers.index(self.sender()) # Looks stupid, but apparently it's not possible to send an argument with a connect() without anonymous functions
        buffer_names = (self.boxes_buffers[0].currentText(), self.boxes_buffers[1].currentText(), self.boxes_buffers[2].currentText())
        # I can't figure out a way to selectively disable and reenable elements in a QComboBox at my current Qt level, so here's a workaround
        # You selected the same buffer twice? Change nothing, reset selection in the box you just tried to change to what it was
        if buffer_names.count(buffer_names[num_changed]) > 1 and buffer_names[num_changed] not in ('None', 'Resonance provider'):
            previous_buffer_name = (self.layouts_buffers[num_changed]).itemAt(1).widget().character.name
            (self.boxes_buffers[num_changed]).setCurrentText(previous_buffer_name)
        elif buffer_names[num_changed] != (self.layouts_buffers[num_changed]).itemAt(1).widget().character.name:
            # Now, I need to replace whichever buffer widget happens to currently be in the slot with the new one - if the new one is a new one
            old_w = (self.layouts_buffers[num_changed]).itemAt(1).widget()
            if buffer_names[num_changed] == 'None':
                new_w = self.placeholder_buffer_widgets[num_changed] # Getting the correct buffer 'None' placeholder widget for this buffer slot
            elif buffer_names[num_changed] == 'Resonance provider': # Resonator character.name depends on their element, but here it is just the box text
                new_w = Widget_resonator() # Make a new one since we don't need to keep track of them at all
            else:
                new_w = self.get_buffer_widget(buffer_names[num_changed])
            old_w.hide()
            (self.layouts_buffers[num_changed]).replaceWidget(old_w, new_w) # Resonator widget should be lost now. I can delete it explicitly too I think, but there's no need
            new_w.show()
            
    # Note: resonator widgets and buffer slot placeholder widgets are processed separately even though their functionality is the same - we don't need to keep track of them beyond there being enough
    # It would be slicker to handle both like the resonator, but I'll leave it as-is as a showcase

    def switch_optimizer(self):
        opt_index = (self.button_opt_build, self.button_opt_buffers).index(self.sender())
        self.layout_optimizers.setCurrentIndex(opt_index)

    def get_current_buffers(self):
        # Return a tuple of currently selected widget_buffer.character's
        # These arcane runes send the character objects placed within the widgets put in the frames by buffer selectors to the hypercarry object. If there is no buffer, the placeholder sends a 'None'
        return (self.layouts_buffers[0].itemAt(1).widget().character, self.layouts_buffers[1].itemAt(1).widget().character, self.layouts_buffers[2].itemAt(1).widget().character)

    def calculate_setup(self):
        output, stats = calculate_output(self.enemy_res, self.layout_hc.currentWidget().character, buffers = self.get_current_buffers())
        self.label_calc_result.setText('Current configuration output is: ' + "{:.1f}".format(output))
        stats_formatted = format_stats(stats)
        self.label_stats_result.setText('Effective stats: ' + stats_formatted)

In [10]:
# Optimizers
class Widget_optimize_build(QtWidgets.QWidget):
    def __init__(self, res_container, hc_container, buffers_container):
        super(Widget_optimize_build, self).__init__()
        self.res_container = res_container
        self.hc_container = hc_container # Giving the optimizer containers from the main widget so they can get the current thing from them
        self.buffers_container = buffers_container
        #
        layout_main = QtWidgets.QVBoxLayout(self)
        layout_checks = QtWidgets.QHBoxLayout()
        self.box_opt_weapon = make_check_box(self, 'Weapon', True)
        self.box_opt_sands = make_check_box(self, 'Sands', True)
        self.box_opt_goblet = make_check_box(self, 'Goblet', True)
        self.box_opt_circlet = make_check_box(self, 'Circlet', True)
        for item in (self.box_opt_weapon, self.box_opt_sands, self.box_opt_goblet, self.box_opt_circlet): layout_checks.addWidget(item)
        layout_checks.addSpacerItem(QtWidgets.QSpacerItem(0, 0, QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Minimum))
        layout_main.addLayout(layout_checks)
        button_optimize = QtWidgets.QPushButton('Optimize', self)
        button_optimize.clicked.connect(self.optimize_build)
        self.label_result = QtWidgets.QLabel('Best configuration is:', self)
        for item in (button_optimize, self.label_result): layout_main.addWidget(item)
        layout_main.addSpacerItem(QtWidgets.QSpacerItem(0, 0, QtWidgets.QSizePolicy.Minimum, QtWidgets.QSizePolicy.Expanding))
        
    def optimize_build(self):
        # Calculate output for weapon and artifact (main stats only) setup variations with the current buffer configuration
        hypercarry = self.hc_container.currentWidget().character
        buffers = (self.buffers_container[0].itemAt(1).widget().character, self.buffers_container[1].itemAt(1).widget().character, self.buffers_container[2].itemAt(1).widget().character)
        #
        if self.box_opt_weapon.isChecked(): weapon_types = list(hypercarry.weapon_types.keys())
        else: weapon_types = (hypercarry.weapon.name,)
        if self.box_opt_sands.isChecked(): sands_types = hypercarry.artifact_types['Sands']
        else: sands_types = (hypercarry.artifacts['Sands'],)
        if self.box_opt_goblet.isChecked(): goblet_types = hypercarry.artifact_types['Goblet']
        else: goblet_types = (hypercarry.artifacts['Goblet'],)
        if self.box_opt_circlet.isChecked(): circlet_types = hypercarry.artifact_types['Circlet']
        else: circlet_types = (hypercarry.artifacts['Circlet'],)
        outputs = list()
        for weapon, sands, goblet, circlet in product(weapon_types, sands_types, goblet_types, circlet_types):
            output, stats = calculate_output(self.res_container.value(), hypercarry, hypercarry.weapon_types[weapon].weapon, (sands, goblet, circlet), buffers)
            outputs.append([output, (weapon, sands, goblet, circlet)])
        best = max(outputs) # Should automatially give the pair with highest output number
        self.label_result.setText('Best configuration is: ' + str(best[1]) + " with the output of " + "{:.1f}".format(best[0]))
    
class Widget_optimize_buffers(QtWidgets.QWidget):
    def __init__(self, res_container, hc_container, get_buffer_widget):
        super(Widget_optimize_buffers, self).__init__()
        self.res_container = res_container
        self.hc_container = hc_container
        self.get_buffer_widget = get_buffer_widget
        #
        layout_main = QtWidgets.QVBoxLayout(self)
        layout_options = QtWidgets.QHBoxLayout()
        layout_num_buffers = QtWidgets.QVBoxLayout()
        label_num_buffers = QtWidgets.QLabel('Number of buffers:', self)
        self.box_num_buffers = make_spin_box(self, (1, 3), 1) # Doesn't need to be connected as we are only looking at it with methods
        layout_num_buffers.addWidget(label_num_buffers)
        layout_num_buffers.addWidget(self.box_num_buffers)
        layout_num_buffers.addSpacerItem(QtWidgets.QSpacerItem(0, 0, QtWidgets.QSizePolicy.Minimum, QtWidgets.QSizePolicy.Expanding))
        layout_available = QtWidgets.QVBoxLayout()
        container_available = QtWidgets.QWidget()
        container_available.setLayout(layout_available)
        self.boxes_available = dict()
        buffer_names = list(MODULE_buffer_types.keys())
        buffer_names.pop()
        buffer_names.pop(0)
        for name in buffer_names:
            self.boxes_available[name] = make_check_box(self, name, True)
            layout_available.addWidget(self.boxes_available[name])
        scroll_available = QtWidgets.QScrollArea()
        scroll_available.setWidget(container_available)
        scroll_available.setWidgetResizable(True)
        scroll_available.setFixedHeight(80)
        layout_options.addLayout(layout_num_buffers)
        layout_options.addWidget(scroll_available)
        button_optimize = QtWidgets.QPushButton('Optimize', self)
        button_optimize.clicked.connect(self.optimize_buffers)
        self.label_result = QtWidgets.QLabel('Best configuration is:', self)
        layout_main.addLayout(layout_options)
        for item in (button_optimize, self.label_result): layout_main.addWidget(item)
        layout_main.addSpacerItem(QtWidgets.QSpacerItem(0, 0, QtWidgets.QSizePolicy.Minimum, QtWidgets.QSizePolicy.Expanding))

    def is_resonance_relevant(self, hypercarry, buffer_names, element):
        if element == 'Anemo' and False: return True
        if element == 'Pyro' and 'ATK' in hypercarry.scales_with: return True
        if element == 'Hydro' and 'HP' in hypercarry.scales_with: return True
        if element == 'Electro' and False: return True
        if element == 'Cryo' and 'CV' in hypercarry.scales_with: return True
        if element == 'Geo' and ('DMG' in hypercarry.scales_with or hypercarry.element == 'Geo'): return True
        if element == 'Dendro' and ('EM' in hypercarry.scales_with or 'Kaedehara Kazuha' in buffer_names): return True
        return False

    def optimize_buffers(self):
        # Calculate output for buffers setup variations and update the result label with the best
        # Ideally, the list of buffers should contain resonators relevant the HC's scalings, then buffers relevant to HC's scalings
        hypercarry = self.hc_container.currentWidget().character
        #
        buffer_names = list()
        for key, value in self.boxes_available.items():
            if value.isChecked() and self.get_buffer_widget(key).character.is_relevant(hypercarry): buffer_names.append(key)
        for element in ('Physical', 'Anemo', 'Pyro', 'Hydro', 'Electro', 'Cryo', 'Geo', 'Dendro'):
            if self.is_resonance_relevant(hypercarry, buffer_names, element):
                buffer_names.append('Resonator: ' + element)
                if hypercarry.element != element: buffer_names.append('Resonator: ' + element)
        outputs = list()
        for buffer_combo in combinations(buffer_names, self.box_num_buffers.value()):
            buffers = list()
            for name in buffer_combo:
                buffers.append(self.get_buffer_widget(name).character) # Combine by names, send character objects - I think this function is unavoidably main widget's because the buffers are stored there
            output, stats = calculate_output(self.res_container.value(), hypercarry, buffers = buffers)
            outputs.append([output, buffer_combo])
        if len(outputs) == 0: self.label_result.setText('No combinations available')            
        else:
            best = max(outputs)
            self.label_result.setText('Best buffers are: ' + str(best[1]) + " with the output of " + "{:.1f}".format(best[0]))

In [11]:
class This_main_window(QtWidgets.QMainWindow):
    def __init__(self):
        super(This_main_window, self).__init__()
        main_widget = Widget_genshin_buff_opt_main()
        self.setCentralWidget(main_widget)
        self.setWindowTitle("GI Hypercarry Optimizer")
        self.setGeometry(50, 100, 750, 600)    
    
if __name__ == '__main__':
    import sys
    # Modified version of the runner code for Jupyter compatibility
    app = QtCore.QCoreApplication.instance()
    if app is None:
        app = QtWidgets.QApplication(sys.argv)
    window = This_main_window()
    window.show()
    app.exec_()
    del app
# End of __main__ section